In [83]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import itertools
import random
import plotly
import plotly.express as px
import scipy.stats as stats
import matplotlib.pyplot as plt
from factor_analyzer.factor_analyzer import FactorAnalyzer
from statsmodels.regression.linear_model import OLS

from src.data import import_data
from src.data.data_class import Data
from src.models.preliminaries import Settings
from src.models.dma import DMA
from src.models.tvp import TVP
from src.models import dm_test


from helper_scripts import variable_groups

from statsmodels.tsa import seasonal

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
data_path = os.path.join(Path().cwd().parent, 'data', 'processed')
# load seasonally adjusted dataset
with open(os.path.join(data_path, 'df_sa.pkl'), 'rb') as f:
    df_sa = pickle.load(f) # load raw data
# "selected_data.csv" contains information about variables names, abbreviations and the required type of transformation
selection = pd.read_csv(os.path.join(data_path, 'selected_data.csv'))
df_sa.describe()

var code,CPI_house_energy,PCI_energy_,HICP_energy,HICP_excl_energy,CPI,deflator_GDP,unemp,employment,GDP,cons_private,...,interest_rate_long,M3,M1,business_conf_manufacturing,business_conf_construct,business_conf_service,business_conf_retail,cons_conf_tendency,business_situation,residential_permits
count,123.000000,108.000000,104.000000,108.000000,123.000000,123.000000,123.000000,122.000000,123.000000,123.000000,...,123.000000,123.000000,123.000000,123.000000,123.000000,107.000000,123.000000,123.000000,123.000000,111.000000
mean,86.044251,87.012686,84.341505,92.755311,0.004721,0.004284,-0.003422,0.001091,0.002326,0.001503,...,3.463803,73.059570,65.051820,-6.727642,-21.869919,16.915888,-15.618699,99.946968,99.928214,29339.023758
std,14.225694,20.686686,20.142079,8.294870,0.004461,0.004434,0.033366,0.004588,0.011852,0.012954,...,2.511013,32.274109,43.145580,13.369695,20.970658,15.694436,10.782244,1.088837,1.342361,12214.439550
min,54.290421,54.877066,50.172049,79.809765,-0.006218,-0.004797,-0.096044,-0.010906,-0.078606,-0.092427,...,-0.604967,28.239989,16.992672,-42.200000,-55.700000,-22.000000,-41.200000,96.952924,95.408918,12903.224314
25%,73.991181,66.273204,65.311428,85.461263,0.002036,0.001382,-0.023105,-0.001753,-0.000176,-0.001626,...,1.381471,41.848292,27.878798,-15.400000,-42.450000,9.850000,-24.200000,99.334636,99.292014,20146.329471
50%,86.943304,93.195680,90.987561,91.996135,0.004041,0.003839,-0.006730,0.000564,0.003102,0.002517,...,3.829682,70.994604,56.696744,-6.100000,-23.000000,16.500000,-17.800000,99.964845,99.990484,25712.133386
75%,99.834497,103.481214,101.972046,99.628004,0.006383,0.005843,0.019722,0.003528,0.005931,0.005418,...,5.147533,93.580208,87.016878,0.800000,-7.150000,22.550000,-6.400000,100.702736,100.968382,37431.426816
max,109.463789,148.621775,115.784535,109.938300,0.019812,0.021539,0.101676,0.016532,0.047462,0.050932,...,8.387854,145.809700,178.189434,25.600000,20.900000,50.100000,13.500000,102.119658,102.336481,64217.291026


In [66]:
# load data with all variables and
params = Settings()                                 # init settings
params.h_fore = 1
params.miss_treatment = 3                           # no treatment for missing variables
params.plag = 4                                     # lag inflation to include later
params.use_x = selection['var code'].to_list()      # specify to use all variables
params.tcodesX = params.get_tcodesX(selection)      # get transformation codes
params.restricted_vars = ['intercept', 'CPI']       # these variables are in all models
params.first_sample_ends = '2004-12-31'             # start of calculation of statistics
# params.print_setting_options()                    # if parameter options should be shown
data = Data(df_sa, params)                          # creates an instance of the Data class that contains the transformed data
df_intercept_CPI = data.X.iloc[:,:(params.plag+params.intercept+1)] # the intercept and annualized CPI+lags are ordered at the beginning of the df - save df to attach it again later
df_sa_trans = data.X.iloc[:,(params.plag+params.intercept+1):]      # remove the vars

%run helper_scripts/variable_groups                 # get variable groups

In [67]:
# specify settings for DMA
params.use_y = ['CPI']

# DMA on differnt lag n_factor combinations

In [68]:
fa_lags_1 = 0
# Instantiate factor analysis object
df_fa = df_sa_trans.drop(infl_vars, axis=1)
n_factors = 16
fa = FactorAnalyzer(n_factors=n_factors, rotation='oblimax')    # quartimax for orthogonal rotation, oblimax for oblique
fa.fit(df_fa)
ev, v = fa.get_eigenvalues()

df_factors = pd.DataFrame(fa.transform(df_fa.fillna(0)), index = df_fa.index)
# rename columns
factor_cols = ['Factor_'+str(i) for i in list(df_factors.columns)]
df_factors.columns = factor_cols

df_fa_post_0_lags = pd.concat((df_intercept_CPI, df_factors), axis=1)


In [69]:
fa_lags_2 = 1
# Instantiate factor analysis object
df_fa = df_sa_trans.drop(infl_vars, axis=1)
n_factors = 8
fa = FactorAnalyzer(n_factors=n_factors, rotation='quartimax')    # quartimax for orthogonal rotation, oblimax for oblique
fa.fit(df_fa)
ev, v = fa.get_eigenvalues()
df_factors = pd.DataFrame(fa.transform(df_fa.fillna(0)), index = df_fa.index)
# rename columns
factor_cols = ['Factor_'+str(i) for i in list(df_factors.columns)]
df_factors.columns = factor_cols
df_factors_with_lag = pd.concat((df_factors, df_factors.shift(1)), axis=1)
factor_cols_lag1 = [i+'_t-1' for i in factor_cols]
df_factors_with_lag.columns = factor_cols+factor_cols_lag1
df_fa_post_1_lag = pd.concat((df_intercept_CPI, df_factors_with_lag), axis=1)
df_fa_post_1_lag = df_fa_post_1_lag.iloc[fa_lags_2:, :]

In [70]:

fa_lags_2 = 3
# Instantiate factor analysis object
df_fa = df_sa_trans.drop(infl_vars, axis=1)
n_factors = 4
fa = FactorAnalyzer(n_factors=n_factors, rotation='oblimax')    # quartimax for orthogonal rotation, oblimax for oblique
fa.fit(df_fa)
ev, v = fa.get_eigenvalues()

df_factors = pd.DataFrame(fa.transform(df_fa.fillna(0)), index = df_fa.index)
# rename columns
factor_cols = ['Factor_'+str(i) for i in list(df_factors.columns)]
df_factors.columns = factor_cols

df_factors_with_lag = pd.concat((df_factors, df_factors.shift(1), df_factors.shift(2), df_factors.shift(3)), axis=1)
factor_cols_lag1 = [i+'_t-1' for i in factor_cols]
factor_cols_lag2 = [i+'_t-2' for i in factor_cols]
factor_cols_lag3 = [i+'_t-3' for i in factor_cols]
df_factors_with_lag.columns = factor_cols+factor_cols_lag1+factor_cols_lag2+factor_cols_lag3
df_fa_post_3_lags = pd.concat((df_intercept_CPI, df_factors_with_lag), axis=1)
df_fa_post_3_lags = df_fa_post_3_lags.iloc[fa_lags_2:, :]

In [71]:
def DMA_steps(params, data, alpha, lamda, save, name, output_path):
    params.alpha = alpha
    params.lamda = lamda
    print(f'(alpha, lambda) = ({alpha}, {lamda})')
    dma = DMA(params, data)
    dma.run_dma()
    dma.forecast_statistics(unit='percent', plot_fe=False, plot_y_fe=False, print_stats=False)
    stats_temp = [params.alpha, params.lamda,
                     dma.MAFE_DMA, dma.MSFE_DMA, dma.BIAS_DMA,
                     dma.MAFE_DMS, dma.MSFE_DMS, dma.BIAS_DMS]
    if save:
        with open(os.path.join(output_path, name+'.pkl'), 'wb') as f:
                pickle.dump(dma, f)
    return dma, stats_temp
def get_stats(dma):
    dma.forecast_statistics(unit='percent', plot_fe=False, plot_y_fe=False, print_stats=False)
    stats_temp = [params.alpha, params.lamda,
                     dma.MAFE_DMA, dma.MSFE_DMA, dma.BIAS_DMA,
                     dma.MAFE_DMS, dma.MSFE_DMS, dma.BIAS_DMS]
    return stats_temp

In [72]:
stats_fe = ['alpha', 'lambda', 'MAFE_DMA', 'MSFE_DMA', 'BIAS_DMA', 'MAFE_DMS', 'MSFE_DMS', 'BIAS_DMS']

In [73]:
params.h_fore = 4

data_fa_h4_0_lags = Data(df_sa, params)
data_fa_h4_0_lags.X = df_fa_post_0_lags
data_fa_h4_0_lags.T = data_fa_h4_0_lags.X.shape[0]
data_fa_h4_0_lags.N = data_fa_h4_0_lags.X.shape[1]

data_fa_h4_1_lags = Data(df_sa, params)
data_fa_h4_1_lags.X = df_fa_post_1_lag
data_fa_h4_1_lags.T = data_fa_h4_1_lags.X.shape[0]
data_fa_h4_1_lags.N = data_fa_h4_1_lags.X.shape[1]
data_fa_h4_1_lags.y_dep = data_fa_h4_1_lags.y_dep[fa_lags_1+1:]    # adjust for lagging

data_fa_h4_3_lags = Data(df_sa, params)
data_fa_h4_3_lags.X = df_fa_post_3_lags
data_fa_h4_3_lags.T = data_fa_h4_3_lags.X.shape[0]
data_fa_h4_3_lags.N = data_fa_h4_3_lags.X.shape[1]
data_fa_h4_3_lags.y_dep = data_fa_h4_3_lags.y_dep[fa_lags_2:]    # adjust for lagging

In [75]:
stats2 = ['MAFE', 'MSFE', 'BIAS']
stats_pd_diff_lags_h4 = pd.DataFrame.from_dict(data={'DMA F.A., h=4, 0 lags, (0.99, 0.99)': stats_fa_h1_0_lags[2:5],
                                            'DMA F.A., h=4, 1 lags, (0.99, 0.99)': stats_fa_h1_1_lags[2:5],
                                               'DMA F.A., h=4, 4 lags, (0.99, 0.99)': stats_fa_h1_3_lags[2:5],
                                               'DMS F.A., h=4, 0 lags, (0.99, 0.99)': stats_fa_h1_0_lags[5:],
                                            'DMS F.A., h=4, 1 lags, (0.99, 0.99)': stats_fa_h1_1_lags[5:],
                                               'DMS F.A., h=4, 4 lags, (0.99, 0.99)': stats_fa_h1_3_lags[5:]},
                                orient='index',
                                columns=stats2)
stats_pd_diff_lags_h4*[100, 100**2, 100]

,MAFE,MSFE,BIAS
"DMA F.A., h=4, 0 lags, (0.99, 0.99)",0.458809,0.663330,0.003420
"DMA F.A., h=4, 1 lags, (0.99, 0.99)",0.462855,0.669115,0.001993
"DMA F.A., h=4, 4 lags, (0.99, 0.99)",0.474190,0.696883,0.008257
"DMS F.A., h=4, 0 lags, (0.99, 0.99)",0.457256,0.662490,0.002561
"DMS F.A., h=4, 1 lags, (0.99, 0.99)",0.463090,0.666844,0.005739
"DMS F.A., h=4, 4 lags, (0.99, 0.99)",0.474825,0.682639,0.012590


In [76]:
print((stats_pd_diff_lags_h4*[100, 100**2, 100]).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &      MAFE &      MSFE &      BIAS \\
\midrule
DMA F.A., h=4, 0 lags, (0.99, 0.99) &  0.458809 &  0.663330 &  0.003420 \\
DMA F.A., h=4, 1 lags, (0.99, 0.99) &  0.462855 &  0.669115 &  0.001993 \\
DMA F.A., h=4, 4 lags, (0.99, 0.99) &  0.474190 &  0.696883 &  0.008257 \\
DMS F.A., h=4, 0 lags, (0.99, 0.99) &  0.457256 &  0.662490 &  0.002561 \\
DMS F.A., h=4, 1 lags, (0.99, 0.99) &  0.463090 &  0.666844 &  0.005739 \\
DMS F.A., h=4, 4 lags, (0.99, 0.99) &  0.474825 &  0.682639 &  0.012590 \\
\bottomrule
\end{tabular}



In [77]:
dm_stats_h4 = stats_pd_diff_lags_h4.copy()
actual_vals = (dma_fa_h1_0_lags.y_dep[dma_fa_h1_0_lags.first_sample_ends:]).tolist()
prediction_dma_0_lags = (dma_fa_h1_0_lags.y_t_DMA[dma_fa_h1_0_lags.first_sample_ends:]).tolist()
prediction_dma_1_lags = dma_fa_h1_1_lags.y_t_DMA[dma_fa_h1_0_lags.first_sample_ends:].tolist()
prediction_dma_3_lags = dma_fa_h1_3_lags.y_t_DMA[dma_fa_h1_0_lags.first_sample_ends:].tolist()
prediction_dms_0_lags = (dma_fa_h1_0_lags.y_t_DMS[dma_fa_h1_0_lags.first_sample_ends:]).tolist()
prediction_dms_1_lags = dma_fa_h1_1_lags.y_t_DMS[dma_fa_h1_0_lags.first_sample_ends:].tolist()
prediction_dms_3_lags = dma_fa_h1_3_lags.y_t_DMS[dma_fa_h1_0_lags.first_sample_ends:].tolist()


pred_dict = {'prediction_dma_0_lags ': prediction_dma_0_lags ,
             'prediction_dma_1_lags ': prediction_dma_1_lags ,
             'prediction_dma_3_lags ': prediction_dma_3_lags ,
             'prediction_dms_0_lags': prediction_dms_0_lags,
             'prediction_dms_1_lags': prediction_dms_1_lags,
             'prediction_dms_3_lags': prediction_dms_3_lags}

dm_stats_h4.index = ['prediction_dma_0_lags', 'prediction_dma_1_lags', 'prediction_dma_3_lags',
                     'prediction_dms_0_lags', 'prediction_dms_1_lags', 'prediction_dms_3_lags']

In [78]:
MAD_res_MAFE = []
MAD_res_MSE = []
for pred in pred_dict.keys():
    print(pred)
    if pred == 'prediction_dms_0_lags':
        MAD_res_MAFE.append(1)
        MAD_res_MSE.append(1)
    else:
        MAD_res_MAFE.append(dm_test.dm_test(actual_vals, pred_dict[pred], prediction_dms_0_lags, crit='MAD', h=1,power=1)[1])
        MAD_res_MSE.append(dm_test.dm_test(actual_vals, pred_dict[pred], prediction_dms_0_lags, crit='MSE', h=1,power=1)[1])
dm_stats_h4.MAFE = MAD_res_MAFE
dm_stats_h4.MSFE = MAD_res_MSE
dm_stats_h4[['MAFE', 'MSFE']]

prediction_dma_0_lags 
prediction_dma_1_lags 
prediction_dma_3_lags 
prediction_dms_0_lags
prediction_dms_1_lags
prediction_dms_3_lags


,MAFE,MSFE
prediction_dma_0_lags,0.070740,0.751303
prediction_dma_1_lags,0.883443,0.871728
prediction_dma_3_lags,0.730108,0.223686
prediction_dms_0_lags,1.000000,1.000000
prediction_dms_1_lags,0.886024,0.839864
prediction_dms_3_lags,0.644572,0.897286


# regression of inflation on all factors

In [125]:
# Instantiate factor analysis object
df_fa = df_sa_trans.drop(infl_vars, axis=1)
n_factors = df_fa.shape[1]
fa = FactorAnalyzer(n_factors=n_factors, rotation='oblimax')    # quartimax for orthogonal rotation, oblimax for oblique
fa.fit(df_fa)
ev, v = fa.get_eigenvalues()
df_factors = pd.DataFrame(fa.transform(df_fa.fillna(0)), index = df_fa.index)

In [126]:
#pd_factor_rsqaured = pd.DataFrame(columns=df_factors.columns)
rsquared = {}
for name, values in df_factors.iteritems():
    df_temp = pd.concat((df_sa.CPI, values), axis=1).dropna()
    model = OLS(endog=df_temp.iloc[:,0], exog=values).fit()
    rsquared[name] = model.rsquared
pd.Series(rsquared, index=df_factors.columns)

0     0.079837
1     0.153182
2     0.077178
3     0.427711
4     0.137743
5     0.186377
6     0.210837
7     0.107773
8     0.142989
9     0.130320
10    0.143593
11    0.196600
12    0.051937
13    0.005309
14    0.202288
15    0.197264
16    0.106508
17    0.206782
18    0.000032
19    0.047204
20    0.000222
21    0.203266
22    0.209117
23    0.143053
24    0.183812
25    0.202385
26    0.203188
27    0.177442
28    0.150588
29    0.147802
30    0.211602
31    0.162739
32    0.167927
33    0.204440
34    0.000000
dtype: float64

In [127]:
print(pd.Series(rsquared, index=df_factors.columns).round(2).to_latex())

\begin{tabular}{lr}
\toprule
{} &     0 \\
\midrule
0  &  0.08 \\
1  &  0.15 \\
2  &  0.08 \\
3  &  0.43 \\
4  &  0.14 \\
5  &  0.19 \\
6  &  0.21 \\
7  &  0.11 \\
8  &  0.14 \\
9  &  0.13 \\
10 &  0.14 \\
11 &  0.20 \\
12 &  0.05 \\
13 &  0.01 \\
14 &  0.20 \\
15 &  0.20 \\
16 &  0.11 \\
17 &  0.21 \\
18 &  0.00 \\
19 &  0.05 \\
20 &  0.00 \\
21 &  0.20 \\
22 &  0.21 \\
23 &  0.14 \\
24 &  0.18 \\
25 &  0.20 \\
26 &  0.20 \\
27 &  0.18 \\
28 &  0.15 \\
29 &  0.15 \\
30 &  0.21 \\
31 &  0.16 \\
32 &  0.17 \\
33 &  0.20 \\
34 &  0.00 \\
\bottomrule
\end{tabular}



In [128]:
factors_R2 = pd.Series(rsquared, index=df_factors.columns)
factors_best = []
factors_R2_temp = factors_R2.copy()
for i in range(8):
    ind = np.argmax(factors_R2_temp)
    print(ind)
    factors_best.append(ind)
    factors_R2_temp.pop(ind)
factors_best

3
29
28
27
26
25
24
23


[3, 29, 28, 27, 26, 25, 24, 23]

In [129]:
df_factors_best = df_factors.iloc[:,factors_best]
df_factors = df_factors_best
df_factors

,3,29,28,27,26,25,24,23
1992-09-30,0.396782,-5.486120,-9.249700,8.201383,-6.859323,13.981377,5.254151,-3.338757
1992-12-31,-0.677819,-6.879979,-9.952047,7.905579,-8.232080,12.211616,5.175147,-4.983818
1993-03-31,2.251620,-6.257700,-9.490395,10.273047,-8.534841,12.959092,4.285806,-4.420032
1993-06-30,0.710671,-6.667990,-9.866440,8.837110,-9.042858,12.472832,5.610175,-4.604277
1993-09-30,-0.039773,-6.613172,-10.059440,8.913000,-7.968029,12.634875,3.692065,-5.749499
...,...,...,...,...,...,...,...,...
2020-12-31,0.571343,1.078664,-0.783378,0.676372,-1.069453,-0.537131,0.092678,-0.464718
2021-03-31,3.558612,-0.537293,0.246248,-1.231944,-0.073097,0.132849,-0.496888,0.861555
2021-06-30,0.400706,0.027273,0.016209,0.030827,0.148216,0.872987,0.544007,0.250484
2021-09-30,3.695218,-0.122740,-0.559278,-0.491034,-0.291802,-0.120722,-0.116066,0.422172


In [130]:
# rename columns
factor_cols = ['Factor_'+str(i) for i in list(df_factors.columns)]
df_factors.columns = factor_cols
df_factors_with_lag = pd.concat((df_factors, df_factors.shift(1)), axis=1)
factor_cols_lag1 = [i+'_t-1' for i in factor_cols]
df_factors_with_lag.columns = factor_cols+factor_cols_lag1
df_fa_post_best = pd.concat((df_intercept_CPI, df_factors_with_lag), axis=1)
df_fa_post_best = df_fa_post_best.iloc[fa_lags_1+1:, :]

In [131]:
data_fa_h4_best = Data(df_sa, params)
data_fa_h4_best.X = df_fa_post_best
data_fa_h4_best.T = data_fa_h4_best.X.shape[0]
data_fa_h4_best.N = data_fa_h4_best.X.shape[1]
data_fa_h4_best.y_dep = data_fa_h4_best.y_dep[fa_lags_1+1:]    # adjust for lagging

In [132]:
dma_fa_h4_best, stats_fa_h4_best = DMA_steps(params, data_fa_h4_best, 0.99, 0.99, False, 'dma_fa_h4_best', None)

(alpha, lambda) = (0.99, 0.99)


100%|██████████| 113/113 [07:56<00:00,  4.22s/it]

DMA finished


In [140]:
stats2 = ['MAFE', 'MSFE', 'BIAS']
stats_pd_best_facts = pd.DataFrame.from_dict(data={'DMA, h=4, 1 lag, (0.99, 0.99)': stats_fa_h4_best[2:5],
                                                   'DMS, h=4, 1 lag, (0.99, 0.99)': stats_fa_h4_best[5:]},
                                orient='index',
                                columns=stats2)
stats_pd_best_facts*[100, 100**2, 100]

,MAFE,MSFE,BIAS
"DMA, h=4, 1 lag, (0.99, 0.99)",0.463535,0.672831,0.007322
"DMS, h=4, 1 lag, (0.99, 0.99)",0.464901,0.681962,0.007550


In [141]:
print((stats_pd_best_facts*[100, 100**2, 100]).round(4).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &    MAFE &    MSFE &    BIAS \\
\midrule
DMA, h=4, 1 lag, (0.99, 0.99) &  0.4635 &  0.6728 &  0.0073 \\
DMS, h=4, 1 lag, (0.99, 0.99) &  0.4649 &  0.6820 &  0.0076 \\
\bottomrule
\end{tabular}



In [135]:
stats_fa_h4_best[5:]

[0.004649013258907723, 6.819619536916893e-05, 7.550358634204778e-05]

In [136]:
6.72830706292638e-05*100**2

0.6728307062926381

In [137]:
3815+1933+12261+1246+12622

31877